<a href="https://colab.research.google.com/github/dtd11/SKT-AI/blob/main/_similarity_BERT_cosine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 1.4 MB/s eta 0:00:00


In [3]:
import os
import json
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import json

# BERT 모델 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.39k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

rust_model.ot:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [4]:
import os
import json

# JSON 파일로부터 사건의 사실 내용과 사건 번호를 추출하는 함수
def load_data_from_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        # 'facts' -> 'bsisFacts' 경로를 따라 사건의 사실 내용 추출 및 문자열로 결합
        case_description = " ".join(data['facts']['bsisFacts'])
        case_number = " ".join(data['info']['caseNo'])
    return case_description, case_number

# 특정 디렉토리 내의 모든 JSON 파일로부터 데이터 로딩
def load_cases_from_directory(directory_paths):
    cases = []
    for directory_path in directory_paths:
        for root, _, files in os.walk(directory_path):
            for filename in files:
                if filename.endswith('.json'):
                    file_path = os.path.join(root, filename)
                    case_description, case_number = load_data_from_json(file_path)
                    cases.append((case_description, case_number))
    return cases

# directory_paths는 여러 디렉토리를 포함하는 리스트입니다.
directory_paths = ['/content/drive/MyDrive/skt 교육/law/civil',
                   '/content/drive/MyDrive/skt 교육/law/criminal']
cases_all = load_cases_from_directory(directory_paths)

In [5]:
cases_all

[('특허권침해소송의 상고심 계속중에 당해 특허발명의 명세서에 대한 정정심결이 확정되었음을 이유로 원심판결을 파기한 사례 특허권침해소송을 심리하는 법원이 특허의 무효심결이 확정되기 이전이라도 특허의 무효 사유 존부에 대한 판단을 할 수 있는지 여부(적극) 및 무효사유가 명백한 특허권에 기초한 금지와 손해배상 등의 청구가 권리남용에 해당하는지 여부(적극)',
  '2 0 0 0 다 6 9 1 9 4'),
 ('파산법 제225조 소정의 파산채권확정소송의 소송가액에 관한 수소법원의 재판에 대한 불복방법(=즉시항고) 파산법 제225조 소정의 파산채권확정소송의 소송가액에 관한 수소법원의 재판에 대하여 이해관계인이 불복하여 항고를 하면서 특별항고로 취급하여 달라는 요청에 따라 수소법원이 이를 특별항고로 보고 대법원에 기록을 송부하였으나, 그와 같은 항고는 즉시항고를 제기한 것으로 취급되어야 한다는 이유로 적법한 관할법원으로 이송한 사례',
  '2 0 0 2 그 7 3'),
 ('민사소송에 있어 읍ㆍ면의 당사자능력 지방자치단체의 하부 행정구역에 불과하여 민사소송법상 당사자능력이 없는 읍을 상대로 한 채권자대위소송에서 정당한 당사자능력자로 당사자 표시를 정정케 하는 조치를 취함이 없이 피대위채권의 존부 또는 본안에 관하여 판단한 원심판결을 파기한 사례',
  '2 0 0 1 다 8 3 2 5 8'),
 ('구 하천법 제75조 소정의 손실을 입은 자가 직접 하천관리청 또는 국가를 상대로 민사소송으로 손실보상을 청구할 수 있는지 여부(소극)',
  '2 0 0 1 다 4 0 8 7 9'),
 ('특정 공장을 신축하기 위하여 일정한 토지를 취득하였다가 사정변경이 생겨 그 토지에 당초 의도한 공장을 건축하지 못한 상태로 있던 중, 그 공장과는 전혀 별개의 다른 공장을 신축하기로 하고 그 다른 공장의 규모 등 필요성에 따라 이미 취득한 토지에 인접한 다른 토지를 취득하여 양 토지상에 공장을 신축한 경우, 인접토지는 구 지방세법 제276조 제1항 제1호 (가)목 소정의 취득세 면제대

In [6]:
sentences = [case[0] for case in cases_all]  # 판례 문장들
sentence_embeddings = model.encode(sentences)

In [13]:
user_input_text = "B라는 사람을 통해서 오피스텔 분양 계약을 했는데, B가 건설사와의 계약 없이 계약금만 받고 사기한 상황. 계약금을 돌려받고 싶지만 B가 사라져서 안되는 상황"
user_input_embedding = model.encode(user_input_text)

# 사용자 입력 문장과 각 판례 문장 간의 유사도 계산
similarities = np.dot(sentence_embeddings, user_input_embedding)

# 상위 3개 유사한 사건의 인덱스 찾기
top_indices = np.argsort(similarities)[-3:][::-1]

# 상위 3개 유사한 사건 출력
for idx in top_indices:
    case_no = cases_all[idx][1]  # 사건 번호
    similarity = similarities[idx]  # 유사도
    case_description = cases_all[idx][0]  # 사건 내용
    print(f'CaseNo: {case_no}, Similarity: {similarity}')
    print(f'Case Description: {case_description}\n')

CaseNo: 9 5 다 5 4 4 6 4, Similarity: 0.2681136131286621
Case Description: 공사의 수급인이 건축중인 도급인 소유의 주택 및 부지를 매수한 후 공사대금을 지급받을 목적으로 제3자에게 그 매매계약상의 지위를 양도하는 계약을 체결한 사안에서, 제반 사정에 비추어 그 양도계약이 소송신탁에 해당하여 무효라고 본 사례

CaseNo: 9 8 다 1 8 5 0 6, Similarity: 0.2485424280166626
Case Description: 건축을 목적으로 매매된 토지에 대하여 건축허가를 받을 수 없어 건축이 불가능하다는 법률적 장애가 매매목적물의 하자에 해당하는지 여부(적극) 및 그 하자의 존부에 관한 판단 기준시(=매매계약 성립시) 무효인 약관조항에 의거하여 계약이 체결된 후 상대방이 약관작성자에게 종전 약관에 따른 계약내용의 이행 및 약정내용을 재차 확인하는 취지의 각서를 작성·교부한 경우, 무효인 약관조항이 유효하게 되거나 위 각서의 내용을 새로운 개별약정으로 볼 수 있는지 여부(소극) 일부 패소의 경우, 소송비용 부담의 결정 방법

CaseNo: 2 0 1 9 다 2 8 5 2 5 7, Similarity: 0.22193677723407745
Case Description: 구 상가건물 임대차보호법 제10조의4 제2항 제3호에서 정한 ‘임대차 목적물인 상가건물을 1년 6개월 이상 영리목적으로 사용하지 아니한 경우’의 의미 및 임대인이 다른 사유로 신규 임대차계약 체결을 거절한 후 사후적으로 1년 6개월 동안 상가건물을 영리목적으로 사용하지 않았다는 사정만으로 위 조항에 따른 정당한 사유로 인정할 수 있는지 여부(소극)



In [14]:
from sklearn.metrics.pairwise import cosine_similarity

# 사용자 입력 문장과 각 판례 문장 간의 코사인 유사도 계산
similarities = cosine_similarity(sentence_embeddings, user_input_embedding.reshape(1, -1))

# 상위 3개 유사한 사건의 인덱스 찾기
top_indices = np.argsort(similarities.flatten())[-3:][::-1]

# 상위 3개 유사한 사건 출력
for idx in top_indices:
    case_no = cases_all[idx][1]  # 사건 번호
    similarity = similarities[idx]  # 유사도
    case_description = cases_all[idx][0]  # 사건 내용
    print(f'CaseNo: {case_no}, Similarity: {similarity}')
    print(f'Case Description: {case_description}\n')

CaseNo: 9 5 다 5 4 4 6 4, Similarity: [0.6103351]
Case Description: 공사의 수급인이 건축중인 도급인 소유의 주택 및 부지를 매수한 후 공사대금을 지급받을 목적으로 제3자에게 그 매매계약상의 지위를 양도하는 계약을 체결한 사안에서, 제반 사정에 비추어 그 양도계약이 소송신탁에 해당하여 무효라고 본 사례

CaseNo: 9 8 다 1 8 5 0 6, Similarity: [0.5438963]
Case Description: 건축을 목적으로 매매된 토지에 대하여 건축허가를 받을 수 없어 건축이 불가능하다는 법률적 장애가 매매목적물의 하자에 해당하는지 여부(적극) 및 그 하자의 존부에 관한 판단 기준시(=매매계약 성립시) 무효인 약관조항에 의거하여 계약이 체결된 후 상대방이 약관작성자에게 종전 약관에 따른 계약내용의 이행 및 약정내용을 재차 확인하는 취지의 각서를 작성·교부한 경우, 무효인 약관조항이 유효하게 되거나 위 각서의 내용을 새로운 개별약정으로 볼 수 있는지 여부(소극) 일부 패소의 경우, 소송비용 부담의 결정 방법

CaseNo: 2 0 2 2 다 2 0 2 4 9 8, Similarity: [0.50672966]
Case Description: 상가건물 임대차보호법 제10조의4에서 정한 권리금 회수 방해로 인한 손해배상책임이 성립하기 위해서는 임차인이 구체적인 인적사항을 제시하면서 신규 임차인이 되려는 자를 임대인에게 주선하였어야 하는지 여부(원칙적 적극) / 임대인이 정당한 사유 없이 임차인이 주선할 신규 임차인이 되려는 자와 임대차계약을 체결할 의사가 없음을 확정적으로 표시한 경우, 임차인이 실제로 신규 임차인을 주선하지 않았더라도 임대인에게 권리금 회수 방해로 인한 손해배상을 청구할 수 있는지 여부(적극) 임대인이 신규 임차인이 되려는 사람과 임대차계약 체결을 위한 협의 과정에서 철거·재건축 계획 및 그 시점을 고지하였다는 사정만으로 상가건물 임대차보호법 제10조의4 제1항 제4호에서 정한 ‘권리금

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

user_input_text = "면허 취득 후 한 달도 안 된 운전자와 택시 운전사가 교통사고를 당했는데, 택시 운전사의 과실 운전으로 인해 손해를 입었지만 보상을 받지 못하는 상황"
user_input_embedding = model.encode(user_input_text)

# 사용자 입력 문장과 각 판례 문장 간의 코사인 유사도 계산
similarities = cosine_similarity(sentence_embeddings, user_input_embedding.reshape(1, -1))

# 상위 10개 유사한 사건의 인덱스 찾기
top_indices = np.argsort(similarities.flatten())[-3:][::-1]

# 상위 10개 유사한 사건 출력
for idx in top_indices:
    case_no = cases_all[idx][1]  # 사건 번호
    similarity = similarities[idx]  # 유사도
    case_description = cases_all[idx][0]  # 사건 내용
    print(f'CaseNo: {case_no}, Similarity: {similarity}')
    print(f'Case Description: {case_description}\n')

CaseNo: 2 0 1 9 다 2 0 8 6 8 7, Similarity: [0.58109707]
Case Description: 갑이 운전하던 을 보험회사의 피보험차량과 병이 운전하던 정 보험회사의 피보험차량이 충돌하는 사고가 발생하여 을 회사의 피보험차량에 동승하고 있던 갑의 남동생 무가 상해를 입자, 정 회사가 피해자인 무 측에 치료비가 포함된 보험금을 지급한 다음 갑과 을 회사를 상대로 구상금을 청구하는 선행소송을 제기하였고, 선행소송에서 갑의 과실비율을 50%로 인정하는 판결이 내려지자, 을 회사가 위 판결에 따라 갑의 보험자로서 갑을 대신하여 정 회사에 구상금을 지급하였는데, 그 후 을 회사가 정 회사를 상대로 ‘자동차보험 구상금 분쟁심의에 관한 상호협정’상 선처리사인 정 회사가 피해자 측 과실이 있는 경우의 구상절차 등에 관한 위 상호협정 시행규약의 조항을 위반하였다며 손해배상을 구한 사안에서, 정 회사에게 손해배상의무가 발생한다고 볼 수 없다고 한 사례 자동차손해배상 보장법 제3조에서 정한 ‘자기를 위하여 자동차를 운행하는 자’와 ‘다른 사람’의 의미 및 자동차 보유자나 사용권자의 배우자나 직계존비속 등 친족이 ‘다른 사람’에 해당하는지 여부(원칙적 적극) / 자동차 운행자나 운전자의 운행 중 과실로 피해를 입은 자가 운행자나 운전자와 신분상 내지 생활관계상 일체를 이루는 관계에 있더라도 그 운행자나 운전자와 피해자 사이에서 운행자나 운전자의 과실이 손해배상책임의 감면 사유가 될 수 있는지 여부(소극)

CaseNo: 2 0 0 1 나 4 7 4 2 1, Similarity: [0.5551383]
Case Description: 자동차등록원부상의 등록 명의를 타인에게 대여한 명의대여자의 운행자성 판단기준 구 자동차손해배상보장법 제14조 제2항, 제28조에 따라 보험자가 손해배상의무를 지기 위하여는 당해 교통사고 피해자가 같은 법 제3조 소정의 '다른 사람'이어야 하는지 여부(적극) 무보험자동차에 동승한 자가 호의동승자로서 그에게 운행지배 및 운

In [16]:
!pip install konlpy

In [18]:
import os
import json
import re
from konlpy.tag import Okt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# 한국어 텍스트 정제 함수
def clean_text(text):
    cleaned_text = re.sub('[^가-힣\s]', '', text)  # 한글과 공백을 제외한 모든 문자 제거
    return cleaned_text

# 불용어 제거 함수
def remove_stopwords(text, stopwords):
    words = text.split()
    result = [word for word in words if word not in stopwords]
    return ' '.join(result)

# 전처리 함수
def preprocess(text, stopwords):
    cleaned_text = clean_text(text)
    okt = Okt()
    tokens = okt.morphs(cleaned_text)  # 형태소 분석기를 사용하여 토큰화
    filtered_text = remove_stopwords(' '.join(tokens), stopwords)
    return filtered_text

# 불용어 리스트
stopwords = ["는", "은", "이", "가", "을", "를", "등", "와", "과"]

# 전처리된 문장들의 임베딩 생성
sentences = [preprocess(case[0], stopwords) for case in cases_all]
sentence_embeddings = model.encode(sentences)

# 사용자 입력 문장 전처리
user_input_text = "B라는 사람을 통해서 오피스텔 분양 계약을 했는데, B가 건설사와의 계약 없이 계약금만 받고 사기한 상황. 계약금을 돌려받고 싶지만 B가 사라져서 안되는 상황"
user_input_text_preprocessed = preprocess(user_input_text, stopwords)
user_input_embedding = model.encode(user_input_text_preprocessed)

# 사용자 입력 문장과 각 판례 문장 간의 코사인 유사도 계산
similarities = cosine_similarity(sentence_embeddings, user_input_embedding.reshape(1, -1))

# 상위 3개 유사한 사건의 인덱스 찾기
top_indices = np.argsort(similarities.flatten())[-3:][::-1]

# 상위 3개 유사한 사건 출력
for idx in top_indices:
    case_no = cases_all[idx][1]  # 사건 번호
    similarity = similarities[idx]  # 유사도
    case_description = cases_all[idx][0]  # 사건 내용
    print(f'CaseNo: {case_no}, Similarity: {similarity}')
    print(f'Case Description: {case_description}\n')

CaseNo: 9 5 다 5 4 4 6 4, Similarity: [0.68499064]
Case Description: 공사의 수급인이 건축중인 도급인 소유의 주택 및 부지를 매수한 후 공사대금을 지급받을 목적으로 제3자에게 그 매매계약상의 지위를 양도하는 계약을 체결한 사안에서, 제반 사정에 비추어 그 양도계약이 소송신탁에 해당하여 무효라고 본 사례

CaseNo: 9 8 다 1 8 5 0 6, Similarity: [0.6297813]
Case Description: 건축을 목적으로 매매된 토지에 대하여 건축허가를 받을 수 없어 건축이 불가능하다는 법률적 장애가 매매목적물의 하자에 해당하는지 여부(적극) 및 그 하자의 존부에 관한 판단 기준시(=매매계약 성립시) 무효인 약관조항에 의거하여 계약이 체결된 후 상대방이 약관작성자에게 종전 약관에 따른 계약내용의 이행 및 약정내용을 재차 확인하는 취지의 각서를 작성·교부한 경우, 무효인 약관조항이 유효하게 되거나 위 각서의 내용을 새로운 개별약정으로 볼 수 있는지 여부(소극) 일부 패소의 경우, 소송비용 부담의 결정 방법

CaseNo: 2 0 2 2 다 2 0 2 4 9 8, Similarity: [0.5934378]
Case Description: 상가건물 임대차보호법 제10조의4에서 정한 권리금 회수 방해로 인한 손해배상책임이 성립하기 위해서는 임차인이 구체적인 인적사항을 제시하면서 신규 임차인이 되려는 자를 임대인에게 주선하였어야 하는지 여부(원칙적 적극) / 임대인이 정당한 사유 없이 임차인이 주선할 신규 임차인이 되려는 자와 임대차계약을 체결할 의사가 없음을 확정적으로 표시한 경우, 임차인이 실제로 신규 임차인을 주선하지 않았더라도 임대인에게 권리금 회수 방해로 인한 손해배상을 청구할 수 있는지 여부(적극) 임대인이 신규 임차인이 되려는 사람과 임대차계약 체결을 위한 협의 과정에서 철거·재건축 계획 및 그 시점을 고지하였다는 사정만으로 상가건물 임대차보호법 제10조의4 제1항 제4호에서 정한 ‘권리금

# 전세사기

In [20]:
# 사회초년생 오피스텔 분양 사기 사
# 사용자 입력 문장 전처리
user_input_text = "B라는 사람을 통해서 오피스텔 분양 계약을 했는데, B가 건설사와의 계약 없이 계약금만 받고 사기한 상황. 계약금을 돌려받고 싶지만 B가 사라져서 안되는 상황"
user_input_text_preprocessed = preprocess(user_input_text, stopwords)
user_input_embedding = model.encode(user_input_text_preprocessed)

# 사용자 입력 문장과 각 판례 문장 간의 코사인 유사도 계산
similarities = cosine_similarity(sentence_embeddings, user_input_embedding.reshape(1, -1))

# 상위 3개 유사한 사건의 인덱스 찾기
top_indices = np.argsort(similarities.flatten())[-4:][::-1]

# 상위 3개 유사한 사건 출력
for idx in top_indices:
    case_no = cases_all[idx][1]  # 사건 번호
    similarity = similarities[idx]  # 유사도
    case_description = cases_all[idx][0]  # 사건 내용
    print(f'CaseNo: {case_no}, Similarity: {similarity}')
    print(f'Case Description: {case_description}\n')

CaseNo: 9 5 다 5 4 4 6 4, Similarity: [0.68499064]
Case Description: 공사의 수급인이 건축중인 도급인 소유의 주택 및 부지를 매수한 후 공사대금을 지급받을 목적으로 제3자에게 그 매매계약상의 지위를 양도하는 계약을 체결한 사안에서, 제반 사정에 비추어 그 양도계약이 소송신탁에 해당하여 무효라고 본 사례

CaseNo: 9 8 다 1 8 5 0 6, Similarity: [0.6297813]
Case Description: 건축을 목적으로 매매된 토지에 대하여 건축허가를 받을 수 없어 건축이 불가능하다는 법률적 장애가 매매목적물의 하자에 해당하는지 여부(적극) 및 그 하자의 존부에 관한 판단 기준시(=매매계약 성립시) 무효인 약관조항에 의거하여 계약이 체결된 후 상대방이 약관작성자에게 종전 약관에 따른 계약내용의 이행 및 약정내용을 재차 확인하는 취지의 각서를 작성·교부한 경우, 무효인 약관조항이 유효하게 되거나 위 각서의 내용을 새로운 개별약정으로 볼 수 있는지 여부(소극) 일부 패소의 경우, 소송비용 부담의 결정 방법

CaseNo: 2 0 2 2 다 2 0 2 4 9 8, Similarity: [0.5934378]
Case Description: 상가건물 임대차보호법 제10조의4에서 정한 권리금 회수 방해로 인한 손해배상책임이 성립하기 위해서는 임차인이 구체적인 인적사항을 제시하면서 신규 임차인이 되려는 자를 임대인에게 주선하였어야 하는지 여부(원칙적 적극) / 임대인이 정당한 사유 없이 임차인이 주선할 신규 임차인이 되려는 자와 임대차계약을 체결할 의사가 없음을 확정적으로 표시한 경우, 임차인이 실제로 신규 임차인을 주선하지 않았더라도 임대인에게 권리금 회수 방해로 인한 손해배상을 청구할 수 있는지 여부(적극) 임대인이 신규 임차인이 되려는 사람과 임대차계약 체결을 위한 협의 과정에서 철거·재건축 계획 및 그 시점을 고지하였다는 사정만으로 상가건물 임대차보호법 제10조의4 제1항 제4호에서 정한 ‘권리금

In [27]:
# 조폭출신 사기꾼‘깡통전세’사례
# 사용자 입력 문장 전처리
user_input_text = "중개업자를 통해 전세계약을 맺었는데 담보 대출과 전세 보증금 합계가 매매가보다 높아 보증금을 돌려받지 못하는 상황"
user_input_text_preprocessed = preprocess(user_input_text, stopwords)
user_input_embedding = model.encode(user_input_text_preprocessed)

# 사용자 입력 문장과 각 판례 문장 간의 코사인 유사도 계산
similarities = cosine_similarity(sentence_embeddings, user_input_embedding.reshape(1, -1))

# 상위 3개 유사한 사건의 인덱스 찾기
top_indices = np.argsort(similarities.flatten())[-4:][::-1]

# 상위 3개 유사한 사건 출력
for idx in top_indices:
    case_no = cases_all[idx][1]  # 사건 번호
    similarity = similarities[idx]  # 유사도
    case_description = cases_all[idx][0]  # 사건 내용
    print(f'CaseNo: {case_no}, Similarity: {similarity}')
    print(f'Case Description: {case_description}\n')

CaseNo: 2 0 1 3 다 2 0 5 1 9 8, Similarity: [0.48632196]
Case Description: 과세관청이 납세자에 대한 체납처분으로서 국내은행 해외지점에 예치된 예금에 대한 반환채권을 대상으로 한 압류처분의 효력(무효)

CaseNo: 2 0 0 6 다 1 4 3 6 3, Similarity: [0.46690023]
Case Description: 구 ‘금융기관부실자산 등의 효율적 처리 및 성업공사의 설립에 관한 법률 시행령’ 제5조 제2항의 입법 취지 및 한국자산관리공사가 금융기관과 사후정산조건부 일괄매입방식으로 부실채권을 일괄인수하기로 계약하면서 그 부실채권의 추심금액이 인수가액을 초과하는 경우 그 초과액 상당을 반환하기로 하는 내용을 포함시키지 않은 경우, 그 계약이 위 규정의 입법 취지에 반하여 무효인지 여부(소극) 한국자산관리공사가 금융기관과 사후정산조건부 일괄매입방식에 의하여 부실채권을 일괄인수하기로 계약하면서 그 계약서 등에 정한 정산 관련 조항들이 약관의 규제에 관한 법률에 위배되어 무효라고 할 수 없다고 한 사례 부실채권의 양도양수계약시 나중에 매입대금을 정산하여 그 차액을 반환하기로 약정하면서 그 차액에 대하여 매입대금의 지급일부터 정산일까지 약정이율에 의한 이자를 가산하여 지급하기로 합의한 경우, 정산일 이후에도 당초의 약정이율에 의한 지연손해금을 가산하여 지급하기로 한 것인지 여부(적극) 파산재단에 관한 소송에서 파산관재인이 여럿인 경우 그 소송의 법적 성격(=필수적 공동소송) 및 공동파산관재인 중 일부가 파산관재인의 자격을 상실한 경우, 남아 있는 파산관재인이 자격을 상실한 파산관재인을 수계하기 위한 절차를 따로 거치지 않고 혼자서 소송행위를 할 수 있는지 여부(적극)

CaseNo: 2 0 1 6 다 2 3 7 9 7 4, Similarity: [0.45826715]
Case Description: 예금거래기본약관에 따라 송금의뢰인이 수취인의 예금계좌에 자금이체를 하여 예금원장에 입금의 기록이 된 

In [28]:
# 사회초년생 노린 건축왕사기
# 사용자 입력 문장 전처리
user_input_text = "건축업자 남 모 씨를 통해 아파트를 전세 계약했으나, 남 모 씨가 이미 다른 사람에게 전세 계약을 맺어 전세 보증금을 가로채고, 이를 돌려받지 못하고 있는 상황"
user_input_text_preprocessed = preprocess(user_input_text, stopwords)
user_input_embedding = model.encode(user_input_text_preprocessed)

# 사용자 입력 문장과 각 판례 문장 간의 코사인 유사도 계산
similarities = cosine_similarity(sentence_embeddings, user_input_embedding.reshape(1, -1))

# 상위 3개 유사한 사건의 인덱스 찾기
top_indices = np.argsort(similarities.flatten())[-4:][::-1]

# 상위 3개 유사한 사건 출력
for idx in top_indices:
    case_no = cases_all[idx][1]  # 사건 번호
    similarity = similarities[idx]  # 유사도
    case_description = cases_all[idx][0]  # 사건 내용
    print(f'CaseNo: {case_no}, Similarity: {similarity}')
    print(f'Case Description: {case_description}\n')

CaseNo: 2 0 1 7 다 2 1 1 4 8 1, Similarity: [0.607344]
Case Description: 자유심증주의의 한계 갑 주식회사가 스스로 건설하여 을 등에게 임대하였다가 분양전환하는 임대아파트의 분양전환가격을 산정하면서 그 산정의 기초가 되는 ‘건설원가’의 한 요소인 ‘실제 건축비’를 위 아파트의 취득세 과세표준이 아닌 건축비 감정 결과를 토대로 인정할 수 있는지 문제 된 사안에서, 위 아파트에 관하여 신고·확정된 취득세 과세표준이 실제 건축비를 반영하는 유력한 증거자료인데도, 합리적인 근거 없이 취득세 과세표준의 증명력을 전부 배척한 다음 건축비 감정 결과를 토대로 실제 건축비를 인정한 원심판단에는 자유심증주의의 한계를 벗어나 필요한 심리를 다하지 않은 위법이 있다고 한 사례 분양전환가격 산정기준에 관한 구 임대주택법 등 관련 법령의 규정이 강행법규인지 여부(적극) 및 위 산정기준에 따른 금액을 초과하는 분양전환가격으로 체결된 분양계약의 효력(=초과하는 범위에서 무효) / 이러한 일부 무효의 법리는 민간 임대사업자가 시장 등의 분양전환승인을 거쳐 분양전환을 한 경우에도 마찬가지로 적용되는지 여부(적극) 공공 임대주택의 분양전환가격을 산정하기 위하여 시장 등이 2곳의 감정평가법인에 의뢰하여 시행한 최초의 감정평가에 대한 이의신청이 있어 재평가를 한 경우, 재평가된 금액을 기초로 분양전환가격을 산정하여야 하는지 여부(원칙적 적극)

CaseNo: 9 5 다 5 4 4 6 4, Similarity: [0.5824422]
Case Description: 공사의 수급인이 건축중인 도급인 소유의 주택 및 부지를 매수한 후 공사대금을 지급받을 목적으로 제3자에게 그 매매계약상의 지위를 양도하는 계약을 체결한 사안에서, 제반 사정에 비추어 그 양도계약이 소송신탁에 해당하여 무효라고 본 사례

CaseNo: 9 2 도 2 9 2 9, Similarity: [0.55865264]
Case Description: 가. 업무방해죄의 보호대상이 되는

In [30]:
# 사회초년생 노린 건축왕사기
# 사용자 입력 문장 전처리
user_input_text = "건축업자 남 모 씨를 통해 아파트를 전세 계약했으나, 남 모 씨가 이미 다른 사람에게 전세 계약을 맺어 전세 보증금을 가로채고, 이를 돌려받지 못하고 있는 상황"
user_input_text_preprocessed = preprocess(user_input_text, stopwords)
user_input_embedding = model.encode(user_input_text_preprocessed)

# 사용자 입력 문장과 각 판례 문장 간의 코사인 유사도 계산
similarities = cosine_similarity(sentence_embeddings, user_input_embedding.reshape(1, -1))

# 상위 3개 유사한 사건의 인덱스 찾기
top_indices = np.argsort(similarities.flatten())[-4:][::-1]

# 상위 3개 유사한 사건 출력
for idx in top_indices:
    case_no = cases_all[idx][1]  # 사건 번호
    similarity = similarities[idx]  # 유사도
    case_description = cases_all[idx][0]  # 사건 내용
    print(f'CaseNo: {case_no}, Similarity: {similarity}')
    print(f'Case Description: {case_description}\n')

CaseNo: 2 0 1 7 다 2 1 1 4 8 1, Similarity: [0.607344]
Case Description: 자유심증주의의 한계 갑 주식회사가 스스로 건설하여 을 등에게 임대하였다가 분양전환하는 임대아파트의 분양전환가격을 산정하면서 그 산정의 기초가 되는 ‘건설원가’의 한 요소인 ‘실제 건축비’를 위 아파트의 취득세 과세표준이 아닌 건축비 감정 결과를 토대로 인정할 수 있는지 문제 된 사안에서, 위 아파트에 관하여 신고·확정된 취득세 과세표준이 실제 건축비를 반영하는 유력한 증거자료인데도, 합리적인 근거 없이 취득세 과세표준의 증명력을 전부 배척한 다음 건축비 감정 결과를 토대로 실제 건축비를 인정한 원심판단에는 자유심증주의의 한계를 벗어나 필요한 심리를 다하지 않은 위법이 있다고 한 사례 분양전환가격 산정기준에 관한 구 임대주택법 등 관련 법령의 규정이 강행법규인지 여부(적극) 및 위 산정기준에 따른 금액을 초과하는 분양전환가격으로 체결된 분양계약의 효력(=초과하는 범위에서 무효) / 이러한 일부 무효의 법리는 민간 임대사업자가 시장 등의 분양전환승인을 거쳐 분양전환을 한 경우에도 마찬가지로 적용되는지 여부(적극) 공공 임대주택의 분양전환가격을 산정하기 위하여 시장 등이 2곳의 감정평가법인에 의뢰하여 시행한 최초의 감정평가에 대한 이의신청이 있어 재평가를 한 경우, 재평가된 금액을 기초로 분양전환가격을 산정하여야 하는지 여부(원칙적 적극)

CaseNo: 9 5 다 5 4 4 6 4, Similarity: [0.5824422]
Case Description: 공사의 수급인이 건축중인 도급인 소유의 주택 및 부지를 매수한 후 공사대금을 지급받을 목적으로 제3자에게 그 매매계약상의 지위를 양도하는 계약을 체결한 사안에서, 제반 사정에 비추어 그 양도계약이 소송신탁에 해당하여 무효라고 본 사례

CaseNo: 9 2 도 2 9 2 9, Similarity: [0.55865264]
Case Description: 가. 업무방해죄의 보호대상이 되는

In [31]:
# 수원 사회초년생 대상 전세사기
# 사용자 입력 문장 전처리
user_input_text = "공인중개사를 통해 아파트를 전세 계약했으나, 공인중개사가 이중 계약을 하여 연락이 끊어졌고, 피해자는 전세금을 돌려받지 못하고 있는 상황"
user_input_text_preprocessed = preprocess(user_input_text, stopwords)
user_input_embedding = model.encode(user_input_text_preprocessed)

# 사용자 입력 문장과 각 판례 문장 간의 코사인 유사도 계산
similarities = cosine_similarity(sentence_embeddings, user_input_embedding.reshape(1, -1))

# 상위 3개 유사한 사건의 인덱스 찾기
top_indices = np.argsort(similarities.flatten())[-4:][::-1]

# 상위 3개 유사한 사건 출력
for idx in top_indices:
    case_no = cases_all[idx][1]  # 사건 번호
    similarity = similarities[idx]  # 유사도
    case_description = cases_all[idx][0]  # 사건 내용
    print(f'CaseNo: {case_no}, Similarity: {similarity}')
    print(f'Case Description: {case_description}\n')

CaseNo: 8 3 도 8 2 3, Similarity: [0.51720685]
Case Description: 가. 아파트를 임대한 소유자가 소유권을 상실한 경우와 기망행위 나. 피고인의 처의 증언의 신빙성 다. 아파트 전매인이 전매시 전매인에게 아파트분양회사 대표이사가 그 분양업무와 관련하여 유죄판결받은 사실의 고지의무 유무

CaseNo: 8 4 모 7 4, Similarity: [0.51618886]
Case Description: 공소제기시 피고인이 공소장에 기재된 주소에 거주하지 않았음에도 검사에게 주소보정을 요구함이 없이 바로 공시송달을 명한 것의 적부(소극)

CaseNo: 9 6 모 5 9, Similarity: [0.49805072]
Case Description: 공소장에 피고인의 사무소 주소가 기재되어 있음에도 불구하고 주거지로 우송한 소송기록접수통지서가 송달불능되자 곧바로 공시송달한 것은 위법하다고 한 사례

CaseNo: 9 2 도 2 9 2 9, Similarity: [0.49347344]
Case Description: 가. 업무방해죄의 보호대상이 되는 업무의 범위 나. 건물 임대인이 구청장의 조경공사 촉구지시에 따라 임대 건물 앞에서 1회적인 조경공사를 하는 데 불과한 경우에는 위 “가”항의 “업무”에 해당되지 않는다고 한 사례



In [33]:
# 광양 100억사
# 사용자 입력 문장 전처리
user_input_text = "공인중개사를 통해 주택담보대출로 근저당이 설정된 노후 아파트를 전세 계약을 했는데, 공인중개사가 이중 계약을 해서 전세보증금을 돌려받지 못하고 있는 상황"
user_input_text_preprocessed = preprocess(user_input_text, stopwords)
user_input_embedding = model.encode(user_input_text_preprocessed)

# 사용자 입력 문장과 각 판례 문장 간의 코사인 유사도 계산
similarities = cosine_similarity(sentence_embeddings, user_input_embedding.reshape(1, -1))

# 상위 3개 유사한 사건의 인덱스 찾기
top_indices = np.argsort(similarities.flatten())[-4:][::-1]

# 상위 3개 유사한 사건 출력
for idx in top_indices:
    case_no = cases_all[idx][1]  # 사건 번호
    similarity = similarities[idx]  # 유사도
    case_description = cases_all[idx][0]  # 사건 내용
    print(f'CaseNo: {case_no}, Similarity: {similarity}')
    print(f'Case Description: {case_description}\n')

CaseNo: 2 0 1 7 다 2 6 9 8 6 2, Similarity: [0.65149987]
Case Description: 국가가 조세채권의 보전을 위하여 납세의무자의 제3자에 대한 채권을 대위하여 행사할 수 있는지 여부(적극) 신탁회사인 갑 주식회사가 을 주식회사와 담보신탁용 부동산관리처분신탁계약을 체결한 후 신탁계약에 근거하여 신탁부동산을 처분하였는데, 신탁계약에서 ‘처분대금 수납 시까지 고지된 당해세’를 우선수익자 등에 우선하여 정산하도록 정하였음을 근거로 을 회사에 대한 조세채권자인 국가가 갑 회사를 상대로 을 회사를 대위하여 정산금채권의 지급을 구한 사안에서, 제반 사정에 비추어 신탁계약에서 정한 ‘처분대금 수납 시까지 고지된 재산세 등 당해세’는 신탁재산과 관련하여 수탁자인 갑 회사에 부과된 당해세만을 의미하므로, 을 회사는 갑 회사에 대한 당해세 상당의 정산금채권을 가지지 못하고 따라서 국가가 을 회사를 대위하여 정산금채권의 지급을 구할 수 없다고 한 사례

CaseNo: 9 5 다 5 4 4 6 4, Similarity: [0.5862737]
Case Description: 공사의 수급인이 건축중인 도급인 소유의 주택 및 부지를 매수한 후 공사대금을 지급받을 목적으로 제3자에게 그 매매계약상의 지위를 양도하는 계약을 체결한 사안에서, 제반 사정에 비추어 그 양도계약이 소송신탁에 해당하여 무효라고 본 사례

CaseNo: 2 0 2 2 다 2 5 5 1 2 6, Similarity: [0.5746614]
Case Description: 주택임대차보호법 제3조의2 제7항에서 정한 금융기관이 임차인으로부터 보증금반환채권을 계약으로 양수하여 양수한 금액의 범위에서 우선변제권을 승계한 다음 경매절차에서 배당요구를 하여 보증금 중 일부를 배당받은 경우, 주택임대차의 대항요건이 존속되는 한 임차인은 보증금반환채권을 양수한 금융기관이 보증금 잔액을 반환받을 때까지 임차주택의 양수인을 상대로 임대차관계의 존속을 주장할 수 있는지 여부(적극)

CaseN

# 교통사고


In [32]:
# 사용자 입력 문장 전처리
user_input_text = "면허 취득 후 한 달도 안 된 운전자와 택시 운전사가 교통사고를 당했는데, 택시 운전사의 과실 운전으로 인해 손해를 입었지만 보상을 받지 못하는 상황"
user_input_text_preprocessed = preprocess(user_input_text, stopwords)
user_input_embedding = model.encode(user_input_text_preprocessed)

# 사용자 입력 문장과 각 판례 문장 간의 코사인 유사도 계산
similarities = cosine_similarity(sentence_embeddings, user_input_embedding.reshape(1, -1))

# 상위 3개 유사한 사건의 인덱스 찾기
top_indices = np.argsort(similarities.flatten())[-4:][::-1]

# 상위 3개 유사한 사건 출력
for idx in top_indices:
    case_no = cases_all[idx][1]  # 사건 번호
    similarity = similarities[idx]  # 유사도
    case_description = cases_all[idx][0]  # 사건 내용
    print(f'CaseNo: {case_no}, Similarity: {similarity}')
    print(f'Case Description: {case_description}\n')

CaseNo: 2 0 1 9 다 2 0 8 6 8 7, Similarity: [0.5596819]
Case Description: 갑이 운전하던 을 보험회사의 피보험차량과 병이 운전하던 정 보험회사의 피보험차량이 충돌하는 사고가 발생하여 을 회사의 피보험차량에 동승하고 있던 갑의 남동생 무가 상해를 입자, 정 회사가 피해자인 무 측에 치료비가 포함된 보험금을 지급한 다음 갑과 을 회사를 상대로 구상금을 청구하는 선행소송을 제기하였고, 선행소송에서 갑의 과실비율을 50%로 인정하는 판결이 내려지자, 을 회사가 위 판결에 따라 갑의 보험자로서 갑을 대신하여 정 회사에 구상금을 지급하였는데, 그 후 을 회사가 정 회사를 상대로 ‘자동차보험 구상금 분쟁심의에 관한 상호협정’상 선처리사인 정 회사가 피해자 측 과실이 있는 경우의 구상절차 등에 관한 위 상호협정 시행규약의 조항을 위반하였다며 손해배상을 구한 사안에서, 정 회사에게 손해배상의무가 발생한다고 볼 수 없다고 한 사례 자동차손해배상 보장법 제3조에서 정한 ‘자기를 위하여 자동차를 운행하는 자’와 ‘다른 사람’의 의미 및 자동차 보유자나 사용권자의 배우자나 직계존비속 등 친족이 ‘다른 사람’에 해당하는지 여부(원칙적 적극) / 자동차 운행자나 운전자의 운행 중 과실로 피해를 입은 자가 운행자나 운전자와 신분상 내지 생활관계상 일체를 이루는 관계에 있더라도 그 운행자나 운전자와 피해자 사이에서 운행자나 운전자의 과실이 손해배상책임의 감면 사유가 될 수 있는지 여부(소극)

CaseNo: 2 0 0 1 나 4 7 4 2 1, Similarity: [0.53025794]
Case Description: 자동차등록원부상의 등록 명의를 타인에게 대여한 명의대여자의 운행자성 판단기준 구 자동차손해배상보장법 제14조 제2항, 제28조에 따라 보험자가 손해배상의무를 지기 위하여는 당해 교통사고 피해자가 같은 법 제3조 소정의 '다른 사람'이어야 하는지 여부(적극) 무보험자동차에 동승한 자가 호의동승자로서 그에게 운행지배 및 운